In [1]:
from dataclasses import dataclass
from pathlib import Path
import os


In [2]:
%pwd

'/Users/saiprakashlikky/Desktop/Projects/ML_projects/Flight_Fare_estimator_Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/saiprakashlikky/Desktop/Projects/ML_projects/Flight_Fare_estimator_Project'

##### Entity declaration dataclass allows to create custom return type

In [5]:
from src.Flight_Fare_estimator_Project.constants import *

In [6]:
from src.Flight_Fare_estimator_Project.utils.common import read_yaml, create_directories

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [9]:
import os
import urllib.request as request
import zipfile
from src.Flight_Fare_estimator_Project import logger
from src.Flight_Fare_estimator_Project.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-24 11:43:49,092:INFO:common:yaml file: config/config.yaml loaded successfully]
[2023-11-24 11:43:49,093:INFO:common:yaml file: params.yaml loaded successfully]
[2023-11-24 11:43:49,094:INFO:common:yaml file: schema.yaml loaded successfully]
[2023-11-24 11:43:49,095:INFO:common:created directory at: artifacts]
[2023-11-24 11:43:49,096:INFO:common:created directory at: artifacts/data_ingestion]
[2023-11-24 11:43:50,139:INFO:1543902713:artifacts/data_ingestion/train.zip download! with following info: 
Connection: close
Content-Length: 502902
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "8e98e35834135e6eb34d5acdb27d4b5bd8108fc630ebeb8eb197d5c24d91b747"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 559C:45FB:53C1B1:68A4B6:6560D2C5
Accept-Ranges: bytes
Date: Fri, 24 Nov 2023 16:43:50